# CBS Week 4 Assessment: Categorization
## Semester 2 2021


In [ ]:
library(tidyverse)
library(mvtnorm)
library(ggpubr)
library(ggrepel)
library(testthat)
library(metR)
options(repr.plot.width=8, repr.plot.height=8)

This notebook is due at 3 pm on August 27. As for all notebooks, a FAQ for this assignment will be maintained on Canvas, and you can find it on the page associated with Week 4. Please make sure that your notebook validates before you submit it --- if your notebook doesn't validate the automated grader may run into issues.

In the week 4 tutorial you looked at one-dimensional categorization problems, and this notebook applies an exemplar model to a two-dimensional categorization problem. We'll use data from an experiment reported by Nosofsky (1989), [Further tests of an exemplar similarity approach to relating identification and categorization](  https://link.springer.com/content/pdf/10.3758/BF03204942.pdf ).  The same paper also describes an exemplar model called the GCM (Generalized Context Model) that is related to but not the same as the exemplar model used in this notebook. The 1989 paper is difficult to read, and a more accessible overview of the GCM is given by  Nosofsky (2011), [The generalized context model: an exemplar model of classification](  
https://nosofsky.cogs.indiana.edu/pubs/Nosofsky_2014_chapter.pdf ), which also includes an application of the model to the 1989 data set (see pp 29 -34). Looking at this 2011 chapter may help you think about some of the questions in this notebook, but this notebook can be completed without reading the chapter.

Nosofsky's experiments used a set of 16 semi-circles that vary on two dimensions -- the size of the semi-circle and the angle of the radius shown. These 16 stimuli can be organized into a grid.


<figure>
  <img src="images/nosofsky_stim.png" alt="nosofsky_stim" style="width:50%">
  <figcaption  class="figure-caption text-center">Figure 1: Stimuli used by Nosofsky (1989). Figure reproduced from the 1989 paper.</figcaption>
</figure>

One of the things that interested Nosofsky is that people might weight these dimensions differently -- e.g. they might be more sensitive to differences in size than differences in angle. Our exemplar model, however, will give the dimensions the same weight. For convenience, we'll scale the dimensions so that along each dimension the stimuli have a mean of zero and a standard deviation of one.

Nosofsky's experiment included 4 conditions that are summarized by the figure below. Don't worry about the small numbers at the top of each square.


<figure>
  <img src="images/nosofsky_conditions.png" alt="nosofsky_conditions" style="width:50%">
  <figcaption  class="figure-caption text-center">Figure 2: The four conditions in the experiment of Nosofsky (1989). Figure reproduced from the 2011 chapter.</figcaption>
</figure>

In each condition participants had to classify all 16 stimuli, but they were only given feedback indicating the true category labels for a subset of the stimuli. For example, in the criss-cross condition the feedback indicated that stimuli 4,7,10 and 13 belonged to category 1 and that stimuli 1,2,15 and 16 belonged to category 2. We're interested in how participants used this feedback to generalize to all of the other stimuli in the grid.

Let's start by loading a file with Nosofsky's data, and making a plot to show the 16 stimuli after scaling both dimensions.


In [ ]:
conditions = c("size", "angle", "crisscross", "diagonal")

ndata <- read_csv("nosofsky89_categorization_data.csv", show_col_types = FALSE) %>% 
  # scale angle and size so that both have zero mean and unit standard deviation
  mutate(angle = scale(angle), size = scale(size)) %>% 
  mutate(x = map2(angle, size, c)) %>% 
  mutate(condition=factor(condition, levels=conditions)) %>% 
  mutate(z = factor(z))

nstim <- ndata %>% 
  select(stim_id, angle, size, x) %>% 
  unique()

stimplot <- nstim  %>% 
  ggplot(aes(x=angle, y=size, label=stim_id)) +
  geom_point() +
  geom_text(nudge_x = 0.12) +
  theme(aspect.ratio=1)

print(stimplot)

As for the week 4 tutorial, we'll use $z$ to denote the category label of an object. We'd like to compute a classification function  $P(z=1|x)$ that we can use to estimate the probability that a new object $x$ anywhere in the 2 dimensional space belongs to category 1.

Bayes rule tells us that $P(z=1|x) \propto P(x|z) P(z=1)$. Let's start with a function for computing the likelihood $P(x|z)$.  An exemplar model estimates $P(x|z=1)$ by dropping a Gaussian kernel (ie a Gaussian bump) on each exemplar of category 1, adding up all these kernels, and renormalizing to make sure that the density integrates to 1.  Because we're looking at a two-dimensional problem, the Gaussian bumps are 2-dimensional Gaussian distributions. We'll assume that these distributions are axis-aligned Gaussians, which means that they can be characterized by a variance along the angle dimension and a variance along the size dimension. To keep things simple we'll set both variances to 1.


In [ ]:
# We'll use an axis-aligned Gaussian kernel where the variance along each dimension (or bandwidth) is 1. Changing this parameter will affect the predictions of the model.

bw <- 1
bw_2d <- diag(c(bw, bw))

# compute p(XNEW|z=CATEGORY)  based on the observations in DATA
p_x_given_z <- function(data, category, xnew, bw = bw_2d){
  newval <- data %>% 
    filter(z == category) %>% 
    mutate(k_val = map_dbl(x, ~dmvnorm(xnew, mean = ., sigma = bw))) %>% 
    summarize(mean_k_val = mean(k_val)) %>% 
    pull()
}

We'll also need a function to compute the prior.  We'll assume that $P(z=1)$ corresponds to the proportion of observed examples that belong to category 1. 



In [ ]:
# compute p(z=CATEGORY) based on the observations in DATA
p_z <- function(data, category){
  sum(data$z == category, na.rm = TRUE)/( sum(!is.na(data$z)) )
}

Now let's set up a function to combine the prior and likelihood in order to compute the posterior $P(z=1|x)$. 

### Question 1 (8 points)

The function here has some errors --  you'll need to fix it so that the columns `p_z1_given_x` and `p_z2_given_z` are computed correctly. These two columns should capture $P(z=1|x)$ and $P(z=2|x)$ respectively.


In [ ]:
# DATA includes exemplars that have been labelled (ie our training data)
# NEWPOINTS includes points that we'd like to classify

classification_curve <- function(data, newpoints, bw = bw_2d) {
  classifications <- newpoints %>% 
    # compute p(z) for all x
    mutate(p_z1 = p_z(data, 1)) %>% 
    mutate(p_z2 = p_z(data, 2)) %>% 
    # compute p(x|z) 
    mutate(p_x_given_z1 = map_dbl(x, ~p_x_given_z(data, 1, ., bw))) %>% 
    mutate(p_x_given_z2 = map_dbl(x, ~p_x_given_z(data, 2, ., bw))) %>% 
    # you'll need to fix the next four lines of code
    # compute p(z|x) up to normalizing constant
    mutate(p_z1_given_x = 0.5) %>% 
    mutate(p_z2_given_x = 0.5) %>% 
    # normalize p(z|x) 
    mutate(p_z1_given_x = 0.5) %>% 
    mutate(p_z2_given_x = 0.5) %>% 
    # YOUR CODE HERE
    stop('No Answer Given!')
}  

We'll use the function just defined to classify all points in a grid that ranges from -2 to 2 along each dimension. We'll start by looking at the distributions $P(x|z=1)$ (red contour plot) and $P(x|z=2)$ (blue contour plot) for each condition.



In [ ]:
newpointgrid <- expand_grid(angle = seq(-2, 2, 0.1), size = seq(-2, 2, 0.1)) %>% 
  mutate(x = map2(angle, size, c)) 

gridclassification <-  ndata %>% 
  group_by(condition) %>% 
  nest() %>% 
  mutate(classifications = map(data, classification_curve, newpointgrid, bw_2d)) %>% 
  unnest(classifications)

densityplot <- gridclassification %>% 
  ggplot(aes(angle, size)) +
  geom_contour(aes(z=p_x_given_z1), color="#F8766D") +
  geom_contour(aes(z=p_x_given_z2), color = "#00BFC4") +
  facet_wrap(~condition) +
  geom_point(data=ndata, aes(color=z))  +
  theme(aspect.ratio=1) 

print(densityplot)

### Question 2 (3 points)

For the criss-cross condition, the distribution $P(x|z=2)$ has two peaks. Where do these peaks like? 


YOUR ANSWER HERE

For each condition we can also plot the classification surface $P(z=1|x)$. The contour on this surface where $P(z=1|x)=0.5$ separates regions that are classified as category 1 from regions that are classified as category 2.

In [ ]:
classificationsurface <- gridclassification %>% 
  ggplot(aes(angle, size)) +
  geom_contour(aes(z=p_z1_given_x), color="black") +
  geom_text_contour(aes(z=p_z1_given_x)) +
  facet_wrap(~condition) +
  geom_point(data=ndata, aes(color=z)) +
  theme(aspect.ratio=1) 

print(classificationsurface)

### Question 3 (8 points)

For each condition, which of the 16 exemplars are assigned to category 1 by the exemplar model? Provide your answer by adding to the lists below. For example, you should edit `size_classified_as_z1` so that it includes the full list of exemplars assigned to category 1 by the model in the size condition (currently this list includes only exemplars 2, 5, 7 and 8). The exemplar labels are based on the plot above that shows the 16 exemplars numbered from 1 to 16.


In [ ]:
size_classified_as_z1<- c(2,5,7,8)
angle_classified_as_z1<- c()
crisscross_classified_as_z1 <- c()
diagonal_classified_as_z1 <- c()

# YOUR CODE HERE
stop('No Answer Given!')

In [ ]:
# this cell contains tests (including some hidden tests) that your answer to Question 3 should pass
expect_gt(length(angle_classified_as_z1), 0) 
expect_gt(length(crisscross_classified_as_z1), 0) 
expect_gt(length(diagonal_classified_as_z1), 0) 

To generate the plots above we applied the exemplar model to every point on a grid that ranges from -2 to 2 along each dimension. Now we'll apply the model to the 16 experimental stimuli only. In each condition, the model assigns each stimulus a probability of belonging to category 1. We'll compare these probabilities with Nosofsky's experimental data -- ie with the proportion of times each stimulus was assigned to category 1 by his participants.

In [ ]:
allconditions <- ndata %>% 
  group_by(condition) %>% 
  nest() %>% 
  mutate(classifications = map(data, classification_curve, nstim, bw_2d))

Now we'll make a dataframe that pairs each stimulus in each condition with a "model probability" (ie $P(z=1|x)$ according to the model) and a "human probability" (ie the proportion of participants who assigned the stimulus to category 1). For each stimulus we'll compute the squared error -- ie the square of the difference between the model probability and the human probability. 



In [ ]:
humanvsmodel <- allconditions %>% 
    unnest(cols=c(classifications)) %>% 
    select(condition, stim_id, p_z1_given_x ) %>% 
    rename(model_p_z1_given_x = p_z1_given_x ) %>% 
    left_join(ndata, by=c("stim_id", "condition")) %>% 
    mutate(squared_error= (model_p_z1_given_x - human_p_z1_given_x)^2) %>% 
    group_by(condition) %>% 
    mutate(sse = round(sum(squared_error), 2)) 

Now we can plot human probabilities against model probabilities. The 4 panels show results for the 4 conditions, and each data point in each panel is labelled with a stimulus number. We've added a diagonal line, and if the model performed perfectly then all of the data points would lie along this line (ie model probabilities would be identical to human probabilities). The "squared error" associated with a point is the square of its vertical distance from the diagonal line.  Each panel is annotated with a SSE (sum squared error) value that represents the sum of the squared errors across all 16 stimuli in a condition.



In [ ]:
sselabs <- humanvsmodel %>% 
   select(condition, sse) %>% 
   unique()
  
humanvsmodelplot <- humanvsmodel %>% 
  ggplot(aes(x=model_p_z1_given_x, y=human_p_z1_given_x, label=stim_id))  +
  geom_point() +
  geom_abline(intercept=0, slope=1) +
  geom_text_repel() +
  geom_text(data=sselabs, aes(x=0.2, y = 0.95, label=paste0("SSE=",sse)))   +
  theme(aspect.ratio=1) +
  facet_wrap(~condition)

print(humanvsmodelplot)

### Question 4 (6 points)

You should see that the overall sum of squared errors (SSE) is higher for the size and angle conditions than for the crisscross and diagonal conditions. Why do you think that the model performance is worse for the size and angle conditions than for the other two conditions? Your answer should contain two components: (i) a statement of a way in which the size and angle conditions differ from the other two conditions (ii) an explanation of why this difference might lead to poorer model performance on the size and angle conditions.



YOUR ANSWER HERE

Now that you've thought about the overall performance for the 4 conditions, let's zoom in and think about responses to individual stimuli.

### Question 5 (3 points)

For which condition and stimulus number is there the greatest discrepancy between model probabilities and human probabilities? Use the `humanvsmodel` dataframe to answer this question.


YOUR ANSWER HERE

### Question 6 (3 points)

Why does the exemplar model predict that the stimulus you just identified probably belongs to category 1?


YOUR ANSWER HERE

### Question 7 (3 points)

Why do you think that people tend to assign the stimulus you just identified to category 2?


YOUR ANSWER HERE

### Question 8 (3 points)

Describe one way in which the model could be adjusted to reduce the discrepancy between model probabilities and human probabilities you identified in Question 5. You don't necessarily need to identify a specific way to adjust the code ---  a high-level description in English will be sufficient.


YOUR ANSWER HERE

Nosofsky's 2011 chapter includes results showing how the GCM performs on the same data set that we've used in this notebook. The plot below shows that the GCM performs very well:

<figure>
  <img src="images/nosofsky_modelresults.png" alt="nosofsky_modelresults" style="width:50%">
  <figcaption  class="figure-caption text-center">Figure 3: Human probabilities plotted against predictions of the GCM. This plot includes results for all four conditions, and is reproduced from the Nosofsky (2011) chapter.</figcaption>
</figure>

The GCM, however, is more complex than the exemplar model considered in this notebook, and includes 3 tuneable parameters rather than 1. For the model in this notebook, the only numerical parameter is the variance (or bandwidth) associated with the Gaussian kernel, which we set to 1.

An important difference between the GCM and the model in this notebook is that the models are defined over different kinds of 2-dimensional spaces. Our model used a *physical* space --- the dimensions of the space were based on physical measurements of the angle and size of the stimuli presented in the experiment. The GCM uses a *psychological* space --- the exemplars are represented as points in a space such that the psychological similarity between two points is a decreasing function of their distance in the space. 

In the 1989 paper Nosofsky estimates the psychological space using data from an identification experiment in which the stimuli are repeatedly presented and participants are asked to identify the size (level 1 through 4) and angle (level 1 through 4) of each stimulus that they see. These data can be used to tell how often one stimulus is confused with another, and these confusion data can be used to estimate the psychological space (stimuli that are often confused should be nearby in the space). The resulting psychological space is shown in the figure below:


<figure>
  <img src="images/nosofsky_mds.png" alt="nosofsky_mds" style="width:50%">
  <figcaption  class="figure-caption text-center">Figure 4: Psychological space used by the GCM. This figure is reproduced from the Nosofsky (1989) chapter.</figcaption>
</figure>

You can see that the stimuli still form a grid, but the grid is a little ragged. For example, the distance between stimuli 13 and 14 is greater than the distance between stimuli 14 and 15. 

### Question 9 (3 points)

For the 16 stimuli in the experiment, people seem to be more sensitive to differences in size than to differences in angle. Explain how we can draw this conclusion from the psychological space represented in Figure 4. 


YOUR ANSWER HERE

Although simple stimuli like the ones Nosofsky used allow for tightly controlled lab experiments, ultimately we would like to explain how people categorize complex real-world objects --- e.g. how radiologists classify X-rays as "normal" or "abnormal", or how dermatologists classify skin lesions as "melanomas" or "non-melanomas." A major challenge when applying psychological models of categorization to real-world problems is to come up with a representation of the underlying psychological space --- ie  the space in which the exemplars are embedded. One possibility is to learn these representations from a large-scale behavioral experiment --- see for example Hebart et al (2020), [Revealing the multidimensional mental representations of natural objects underlying human similarity judgements ]( https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7666026/ ) . Another promising approach is to use representations derived from convolutional neural networks --- see for example  Battleday et al (2020), [ Capturing human categorization of natural images by combining deep networks and cognitive models ]( https://www.nature.com/articles/s41467-020-18946-z.pdf?origin=ppub ). You'll have a chance to learn about neural networks in the second module of this course.